In [15]:
import numpy as np
import pandas as pd

In [16]:
import torch
from sklearn import preprocessing
torch.manual_seed(1)

In [17]:
train_df = pd.read_csv("./MNIST손글씨분류_data/train.csv")
test_df = pd.read_csv("./MNIST손글씨분류_data/test.csv")
submit_df = pd.read_csv("./MNIST손글씨분류_data/sample_submit.csv")
print(train_df.head())
print(test_df.head())
print(submit_df.head())

   label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0      5    0    0    0    0    0    0    0    0    0  ...      0      0   
1      0    0    0    0    0    0    0    0    0    0  ...      0      0   
2      3    0    0    0    0    0    0    0    0    0  ...      0      0   
3      3    0    0    0    0    0    0    0    0    0  ...      0      0   
4      6    0    0    0    0    0    0    0    0    0  ...      0      0   

   28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  
0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0  
3      0      0      0      0      0      0      0      0  
4      0      0      0      0      0      0      0      0  

[5 rows x 785 columns]
   1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  1x10  ...  28x19  28x20  \
0    0    0    0    0    0    0    0    0    0     0  ...      0      0   
1    0   

In [21]:
y_train = torch.LongTensor(train_df['label'])
x_train = torch.FloatTensor(train_df.drop(['label'],axis=1).to_numpy())
print(y_train.shape)
print(x_train.shape)

torch.Size([60000])
torch.Size([60000, 784])


In [22]:
class_num = 10

w = torch.zeros((784,class_num),requires_grad = True)
b = torch.zeros(1,requires_grad=True)

optimizer = torch.optim.SGD([w,b],lr=1e-5)

epochs = 1000

for epoch in range(epochs + 1):
    hypothesis = torch.nn.functional.softmax(x_train.matmul(w)+b,dim=1)
    y_one_hot = torch.zeros(len(y_train),class_num)
    y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
    cost = (y_one_hot * torch.log(torch.nn.functional.softmax(hypothesis,dim=1))).sum(dim=1).mean()
    
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if(epoch % 100 == 0):
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, epochs, cost.item()
        ))

Epoch    0/1000 Cost: -2.302585
Epoch  100/1000 Cost: -2.449378
Epoch  200/1000 Cost: -2.453475
Epoch  300/1000 Cost: -2.454796
Epoch  400/1000 Cost: -2.455494
Epoch  500/1000 Cost: -2.455941
Epoch  600/1000 Cost: -2.456259
Epoch  700/1000 Cost: -2.456501
Epoch  800/1000 Cost: -2.456693
Epoch  900/1000 Cost: -2.456851
Epoch 1000/1000 Cost: -2.456984


In [25]:
# 학습된 W,b를 통한 클래스 예측
hypothesis = torch.nn.functional.softmax(x_train.matmul(w) + b, dim=1) # or .mm or @
#argmax -> 리스트 중 제일 큰 값의 인덱스로 변환 
predict = torch.argmax(hypothesis, dim=1)
print(hypothesis)
print(predict)
print(y_train)
# 정확도 계산 
correct_prediction = predict.float() == y_train
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

tensor([[8.1475e-01, 1.8501e-01, 2.6809e-05,  ..., 2.5180e-05, 2.2281e-05,
         3.4305e-05],
        [1.7191e-06, 9.9999e-01, 8.4493e-07,  ..., 7.0332e-07, 5.7109e-07,
         6.8410e-07],
        [1.5310e-03, 9.9845e-01, 2.7047e-06,  ..., 3.6998e-06, 2.1613e-06,
         3.1009e-06],
        ...,
        [4.1461e-02, 9.5842e-01, 2.4041e-05,  ..., 8.7910e-06, 1.3191e-05,
         9.8254e-06],
        [2.0523e-07, 1.0000e+00, 1.8875e-07,  ..., 3.0944e-07, 3.0007e-07,
         2.9082e-07],
        [1.5918e-03, 9.9818e-01, 3.1776e-05,  ..., 4.6183e-05, 2.3135e-05,
         2.8768e-05]], grad_fn=<SoftmaxBackward>)
tensor([0, 1, 1,  ..., 1, 1, 1])
tensor([5, 0, 3,  ..., 9, 4, 6])
tensor([False, False, False,  ..., False, False, False])
The model has an accuracy of 0.14% for the training set.


In [26]:
x_test = torch.FloatTensor(test_df.to_numpy())

In [28]:
hypothesis = torch.nn.functional.softmax(x_test.matmul(w) + b, dim=1) # or .mm or @
#argmax -> 리스트 중 제일 큰 값의 인덱스로 변환 
predict = torch.argmax(hypothesis, dim=1)

In [29]:
submit_df['label'] = predict
submit_df.head()

,id,label
0,1,1
1,2,0
2,3,1
3,4,1
4,5,1


In [30]:
submit_df.to_csv("./MNIST손글씨분류_data/submission.csv")